# Placing an order

## Check for time consistency with server

In [4]:
# check for condition supplied in guide: 
# https://bybit-exchange.github.io/docs/v5/guide

'''
Please make sure that the timestamp parameter adheres to the following rule:
server_time - recv_window <= timestamp < server_time + 1000
server_time stands for Bybit server time, which can be queried via the Server Time endpoint.
'''

url = "https://api-testnet.bybit.com/v5/market/time"
req = requests.request("GET", url)
res = req.json()
print(res["time"])#.result.time)
print(int(time.time() * 1000))
print(res["time"]-5000 <= int(time.time() * 1000) and int(time.time() * 1000) < res["time"])

1717410203113
1717410201719
True


# directly from bybit examples

## important info
It seems that there are four server spaces at bybit: mainnet, testnet, demo and demo-testnet. The Demo space seems to be the most fitting for the purpose I intend to use it, namely letting the bot run a little. See for further explanation: https://www.bybit.com/en/help-center/article/FAQ-Demo-Trading


**other infos**

get acc balance: https://bybit-exchange.github.io/docs/api-explorer/v5/account/wallet

integration guide: https://bybit-exchange.github.io/docs/v5/guide

demo von dem untenstehendem code: https://github.com/bybit-exchange/api-usage-examples/blob/master/V5_demo/api_demo/Encryption_HMAC.py

liste von bybit errors: https://bybit-exchange.github.io/docs/v5/error

In [16]:
import requests
import time
import hashlib
import hmac
import uuid

# HMAC KEY FOR TESTNET > idk NET
#api_key = "vgcrg9nWw7owVYEDdc"
#secret_key = "YPz7eb0YzhOpa9s2yMEj3P4UdXrmSEtqRDCP"

# TESTNET
# HMAC KEY FOR READ ONLY WALLET BALANCE
# api_key = "kO4Wf9sdmWk0IeFRBc"
# secret_key = "ZxyPUKE893Alxq7aFdNSZUo5xYGrcp5hKgNO"

# DEMO NET
api_key = "GMUtVbMzRhohkOagoW"
secret_key = "vexur1jjKy5AiQE0rXMhVYbjRFRy6jVxhIhf"

httpClient=requests.Session()
recv_window=str(10000)
# url = "https://api-testnet.bybit.com" # Testnet endpoint
url = "https://api-demo.bybit.com" # DEMO endpoint

def HTTP_Request(endPoint,method,payload,Info):
    global time_stamp
    time_stamp=str(int(time.time() * 10 ** 3))
    signature=genSignature(payload)
    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    }
    if(method=="POST"):
        response = httpClient.request(method, url+endPoint, headers=headers, data=payload)
    else:
        response = httpClient.request(method, url+endPoint+"?"+payload, headers=headers)
    print(response)
    print(response.text)
    print(Info + " Elapsed Time : " + str(response.elapsed))
    return response#.json()

def genSignature(payload):
    param_str= str(time_stamp) + api_key + recv_window + payload
    hash = hmac.new(bytes(secret_key, "utf-8"), param_str.encode("utf-8"),hashlib.sha256)
    signature = hash.hexdigest()
    return signature

# pybit functions

In [4]:
import time
import hmac
import hashlib
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

def generate_signature(use_rsa_authentication, secret, param_str):
    def generate_hmac():
        hash = hmac.new(
            bytes(secret, "utf-8"),
            param_str.encode("utf-8"),
            hashlib.sha256,
        )
        return hash.hexdigest()

    def generate_rsa():
        hash = SHA256.new(param_str.encode("utf-8"))
        encoded_signature = base64.b64encode(
            PKCS1_v1_5.new(RSA.importKey(secret)).sign(
                hash
            )
        )
        return encoded_signature.decode()

    if not use_rsa_authentication:
        return generate_hmac()
    else:
        return generate_rsa()

In [7]:
# pybit functions: 

payload = "&".join(
            [
                str(k) + "=" + str(v)
                for k, v in sorted(parameters.items())
                if v is not None
            ]

param_str = str(timestamp) + self.api_key + str(recv_window) + payload

    return generate_signature(
        self.rsa_authentication, self.api_secret, param_str
    )


def auth(self, payload, recv_window, timestamp):
    """
    Prepares authentication signature per Bybit API specifications.
    """

    if self.api_key is None or self.api_secret is None:
        raise PermissionError("Authenticated endpoints require keys.")

    param_str = str(timestamp) + self.api_key + str(recv_window) + payload

    return generate_signature(
        self.rsa_authentication, self.api_secret, param_str
    )
    
signature = auth(
                payload=req_params,
                recv_window=recv_window,
                timestamp=timestamp,
            )


In [17]:
print(api_key)
#Get Wallet Balance
endpoint="/v5/account/wallet-balance"
method="GET"
params='accountType=UNIFIED'
res = HTTP_Request(endpoint,method,params,"UNIFIED")
res


GMUtVbMzRhohkOagoW
<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"list":[{"totalEquity":"172866.328628","accountIMRate":"0","totalMarginBalance":"99990.5","totalInitialMargin":"0","accountType":"UNIFIED","totalAvailableBalance":"99990.5","accountMMRate":"0","totalPerpUPL":"0","totalWalletBalance":"99990.5","accountLTV":"0","totalMaintenanceMargin":"0","coin":[{"availableToBorrow":"","bonus":"0","accruedInterest":"0","availableToWithdraw":"50000","totalOrderIM":"0","equity":"50000","totalPositionMM":"0","usdValue":"49998.7","unrealisedPnl":"0","collateralSwitch":true,"spotHedgingQty":"0","borrowAmount":"0.000000000000000000","totalPositionIM":"0","walletBalance":"50000","cumRealisedPnl":"0","locked":"0","marginCollateral":true,"coin":"USDC"},{"availableToBorrow":"","bonus":"0","accruedInterest":"0","availableToWithdraw":"1","totalOrderIM":"0","equity":"1","totalPositionMM":"0","usdValue":"69105.04055","unrealisedPnl":"0","collateralSwitch":false,"spotHedgingQty":"0","borrowAmoun

<Response [200]>

In [ ]:
#Create Order
endpoint="/v5/order/create"
method="POST"
orderLinkId=uuid.uuid4().hex
params='{"category":"linear","symbol": "BTCUSDT","side": "Buy","positionIdx": 0,"orderType": "Limit","qty": "0.001","price": "60000","timeInForce": "GTC","orderLinkId": "' + orderLinkId + '"}'
HTTP_Request(endpoint,method,params,"Create")

In [ ]:
'''
#Get unfilled Orders
endpoint="/v5/order/realtime"
method="GET"
params='category=linear&settleCoin=USDT'
HTTP_Request(endpoint,method,params,"UnFilled")


#Cancel Order
endpoint="/v5/order/cancel"
method="POST"
params='{"category":"linear","symbol": "BTCUSDT","orderLinkId": "'+orderLinkId+'"}'
HTTP_Request(endpoint,method,params,"Cancel")
'''

# first tries and other stuff

In [23]:
import time
import hashlib
import hmac
import requests

# Replace these with your API key and secret
# api_key_testnet = "yeOt7DrchkqM9r7sQy"
# api_secret_testnet = "JJHVxNX2wc7di918pwjxSJemJd6j1bsAVRmO"

# hmac api keys
api_key = "vgcrg9nWw7owVYEDdc"
api_secret = "YPz7eb0YzhOpa9s2yMEj3P4UdXrmSEtqRDCP"


def sign(secret,payload):
    return hmac.new(bytes(secret, 'utf-8'), bytes(payload, 'utf-8'), hashlib.sha256).hexdigest()

def place_order(api_key, api_secret, side, price,take_profit,stop_loss):
    
    url = 'https://api-testnet.bybit.com/v5/order/create'

    params = {
        #'timestamp': int(time.time() * 1000),
        #'api_key': api_key,
        #'recv_window': 5000,
        'category' : 'inverse',
        'symbol' : 'BTCUSDT',
        #'isLeverage' : '0',
        'side': side,
        'orderType': 'Limit',
        'qty': '0.003',
        'price': price,
        #'time_in_force': 'GTC',
        #'takeProfit' : take_profit,
        #'tpLimitPrice' : take_profit,
        #'tpOrderType' : 'Limit',
        #'stopLoss' : stop_loss,
        #'slLimitPrice' : stop_loss,
        #'slOrderType' : 'Limit',
        #'tpslMode' : 'Partial',
    }

    # Create the query string
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])

    print(params.items())
    ts = str(int(time.time() * 1000))
    
    # Sign the query string
    params['sign'] = sign(ts, api_secret,query_string)

    # Send the request
    response = requests.post(url, data=params)
    
    return response

# Example usage:
symbol = 'BTCUSDT'
side = 'Buy'
price = 70000  # Limit price
take_profit = 71000
stop_loss = 68000


order_response = place_order(api_key, api_secret, side, price,take_profit,stop_loss)

print(order_response)
print(order_response.json())


dict_items([('category', 'inverse'), ('symbol', 'BTCUSDT'), ('side', 'Buy'), ('orderType', 'Limit'), ('qty', '0.003'), ('price', 70000)])


TypeError: new() takes from 1 to 3 positional arguments but 4 were given

In [58]:
import requests

url = "https://api-testnet.bybit.com/v5/order/create"
timestamp = int(time.time() * 1000)

payload = "{\n  \"category\": \"linear\",\n  \"symbol\": \"BTCUSDT\",\n  \"side\": \"Buy\",\n  \"orderType\": \"Limit\",\n  \"qty\": \"0.002\",\n  \"price\": \"70000\",\n  \"triggerPrice\": null,\n  \"triggerDirection\": null,\n  \"triggerBy\": null,\n  \"orderFilter\": null,\n  \"orderIv\": null,\n  \"timeInForce\": \"GTC\",\n  \"positionIdx\": 0,\n  \"orderLinkId\": \"test-xx1\",\n  \"takeProfit\": null,\n  \"stopLoss\": null,\n  \"tpTriggerBy\": null,\n  \"slTriggerBy\": null,\n  \"reduceOnly\": false,\n  \"closeOnTrigger\": false,\n  \"smpType\": null,\n  \"mmp\": null,\n  \"tpslMode\": null,\n  \"tpLimitPrice\": null,\n  \"slLimitPrice\": null,\n  \"tpOrderType\": null,\n  \"slOrderType\": null\n}"
headers = {
    "Content-Type": "application/json",
    'X-BAPI-API-KEY': str(api_key_testnet),
    'X-BAPI-TIMESTAMP': str(timestamp),
    'X-BAPI-RECV-WINDOW': '20000',
    'X-BAPI-SIGN': '8b96a595c486b7b5ed0d7ad66f001b4d206026e761d73406080f9cae57cd9ecd'
}

  headers = {
                    "Content-Type": "application/json",
                    "X-BAPI-API-KEY": api_key,
                    "X-BAPI-SIGN": signature,
                    "X-BAPI-SIGN-TYPE": "2",
                    "X-BAPI-TIMESTAMP": str(timestamp),
                    "X-BAPI-RECV-WINDOW": str(recv_window),
                }

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"retCode":10003,"retMsg":"API key is invalid.","result":{},"retExtInfo":{},"time":1717364367559}
